In [11]:
# Let's load the package
import numpy as np
from numba import cuda

In [3]:
# let's prepare the data
np.random.seed(42)
start = np.random.randint(1,10,(1024,1024))
symmetry = np.tril(start,k=0) + np.tril(start,k=-1).T  # [1024,1024]

In [5]:
# let's design the necessary sequential version functions of MCL algorithm
def add_self_loop(adjacency,loop_value):
    for i in range(adjacency.shape[0]):
        adjacency[i,i] += loop_value
    return adjacency

def l1_normalization_like(adjacency):
    result = np.empty(adjacency.shape)
    for j in range(adjacency.shape[1]):
        result[:,j] = adjacency[:,j] / np.sum(adjacency[:,j])
    return result

def expand(adjacency,expansion):
    result = adjacency ** expansion
    return result

def inflate(adjacency,inflation):
    result = l1_normalization_like(adjacency**2)
    return result

def iterate(adjacency,expansion=2,inflation=2):
    result = inflate(expand(adjacency,expansion),inflation)
    return result

def allclose(matrix1,matrix2,atol=1e-08,rtol=1e-05):
    c = np.abs(matrix1-matrix2) - (atol + rtol*np.abs(matrix2))
    if np.max(c) <= 0:
        return (True,c)
    else:
        return (False,c)

In [56]:
%%timeit
# let's run the sequential MCL
adjacency = symmetry

# add self-loop (Optional)

# l1 norm of each column, distort the symmetry of the original adjacency matrix
adjacency = l1_normalization_like(adjacency)

for i in range(1000000000):
    # iterate (expand + inflate)
    adjacency_old = np.copy(adjacency)
    adjacency_new = iterate(adjacency)
    # check convergence
    isConverged,c_matrix = allclose(adjacency_old,adjacency_new)
    if isConverged:
        break
    else:
        adjacency = adjacency_new

172 ms ± 3.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
# Now let's dive into the cuda version, the parallel version of MCL
# implement outline:
# 1. write three kernel function: expand, inflate, check_converge
# 2. run the MCL from the host

In [12]:
# Kernel function1 : expand
@cuda.jit
def gpu_expand(in_matrix,out_matrix):  # 2-d block,threads
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bx = cuda.blockIdx.x
    by = cuda.blockIdx.y
    blocksize_x = cuda.blockDim.x
    blocksize_y = cuda.blockDim.y
    pos_x = tx + blocksize_x * bx
    pos_y = ty + blocksize_y * by
    if pos_x <= in_matrix.shape[0] and pos_y <= in_matrix.shape[1]:
        out_matrix[pos_x,pos_y] = in_matrix[pos_x,pos_y] ** 2
        

In [36]:
# kernel function2: inflation
@cuda.jit
def gpu_inflate(in_matrix,out_matrix,final_matrix):  # 1-d block, threads
    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x
    blocksize_x = cuda.blockDim.x
    pos_x =  tx + blocksize_x * bx
    if pos_x <= in_matrix.shape[1]:
        for i in range(out_matrix.shape[0]):
            out_matrix[i,pos_x] = in_matrix[i,pos_x] ** 2
        sum_ = 0
        for i in range(out_matrix.shape[0]):
            sum_ += out_matrix[:,pos_x][i]
        for i in range(out_matrix.shape[0]):
            final_matrix[i,pos_x] = out_matrix[i,pos_x] / sum_  # within-column normalization
        

In [48]:
# kernel function3: check_converge
@cuda.jit
def gpu_check_converge(old_matrix,new_matrix,result_matrix):
    atol = 1e-5
    rtol = 1e-8
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bx = cuda.blockIdx.x
    by = cuda.blockIdx.y
    blocksize_x = cuda.blockDim.x
    blocksize_y = cuda.blockDim.y
    pos_x = tx + blocksize_x * bx
    pos_y = ty + blocksize_y * by
    if pos_x <= old_matrix.shape[0] and pos_y <= old_matrix.shape[1]:
        c = abs(new_matrix[pos_x,pos_y] - old_matrix[pos_x,pos_y]) - (atol + rtol*abs(old_matrix[pos_x,pos_y]))
        if c <= 0: 
            result_matrix[pos_x,pos_y] = True
        else:
            result_matrix[pos_x,pos_y] = False
    
        

In [57]:
%%timeit
# let's run the gpu version of MCL

adjacency = symmetry
adjacency = l1_normalization_like(adjacency)

blocks_per_grid = (32,32)
threads_per_block = (32,32)

for n in range(4):
    # expand
    out_matrix = np.empty([1024,1024])
    d_adjacency = cuda.to_device(adjacency)
    d_out_matrix = cuda.to_device(out_matrix)

    gpu_expand[blocks_per_grid,threads_per_block](d_adjacency,d_out_matrix)
    d_out_matrix.copy_to_host(out_matrix)

    # inflate
    out_matrix1 = np.empty([1024,1024])
    final_matrix = np.empty([1024,1024])
    d_out_matrix = cuda.to_device(out_matrix)
    d_out_matrix1 = cuda.to_device(out_matrix1)
    d_final_matrix = cuda.to_device(final_matrix)
    gpu_inflate[blocks_per_grid,threads_per_block](d_out_matrix,d_out_matrix1,d_final_matrix)
    d_final_matrix.copy_to_host(final_matrix)

    # check convergence
    d_adjacency = cuda.to_device(adjacency)
    d_final_matrix = cuda.to_device(final_matrix)
    result_matrix = np.empty([1024,1024])
    d_result_matrix = cuda.to_device(result_matrix)
    d_final_matrix = cuda.to_device(final_matrix)
    gpu_check_converge[blocks_per_grid,threads_per_block](d_adjacency,d_final_matrix,d_result_matrix)
    d_result_matrix.copy_to_host(result_matrix)
    cond = np.all(result_matrix)
    if cond == True:
        break
    else:
        adjacency = result_matrix


244 ms ± 2.36 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


999
